## GITHUB Topic and Repository Scraping Project

In [202]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [2]:
# Get URL for scraping main topics
response= requests.get('https://github.com/topics')

In [3]:
response.status_code

200

In [4]:
len(response.text)

152059

In [8]:
page_content=response.text

In [217]:
# Get target page content on local machine
with open('webpage.html', 'w', encoding="utf-8") as f:
    f.write(page_content)

In [14]:
# Create BeautifulSoup Object for parsing 
soup= BeautifulSoup(page_content, 'html.parser')

In [15]:
type(soup)

bs4.BeautifulSoup

### Scraping Main topic .their Description & Topic URL in start

In [218]:
topic_title_tags= soup.find_all('p', {'class': 'f3 lh-condensed mb-0 mt-1 Link--primary'})

In [31]:
topic_title_tags[:5]

[<p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Ajax</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Algorithm</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Amp</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Android</p>]

In [26]:
topic_title_desc= soup.find_all('p', {'class', 'f5 color-fg-muted mb-0 mt-1'})

In [32]:
topic_title_desc[:5]

[<p class="f5 color-fg-muted mb-0 mt-1">
           3D modeling is the process of virtually developing the surface and structure of a 3D object.
         </p>,
 <p class="f5 color-fg-muted mb-0 mt-1">
           Ajax is a technique for creating interactive web applications.
         </p>,
 <p class="f5 color-fg-muted mb-0 mt-1">
           Algorithms are self-contained sequences that carry out a variety of tasks.
         </p>,
 <p class="f5 color-fg-muted mb-0 mt-1">
           Amp is a non-blocking concurrency library for PHP.
         </p>,
 <p class="f5 color-fg-muted mb-0 mt-1">
           Android is an operating system built by Google designed for mobile devices.
         </p>]

In [39]:
topics_url= soup.find_all('a', {'class': 'no-underline flex-1 d-flex flex-column'})

In [44]:
topic0_url="https://github.com" + topics_url[0]['href']

In [53]:
topic_title=[]

for title in topic_title_tags:
    topic_title.append(title.text)
print(topic_title)

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android', 'Angular', 'Ansible', 'API', 'Arduino', 'ASP.NET', 'Atom', 'Awesome Lists', 'Amazon Web Services', 'Azure', 'Babel', 'Bash', 'Bitcoin', 'Bootstrap', 'Bot', 'C', 'Chrome', 'Chrome extension', 'Command line interface', 'Clojure', 'Code quality', 'Code review', 'Compiler', 'Continuous integration', 'COVID-19', 'C++']


In [55]:
topic_desc=[]

for title in topic_title_desc:
    topic_desc.append(title.text.strip())
print(topic_desc[:5])

['3D modeling is the process of virtually developing the surface and structure of a 3D object.', 'Ajax is a technique for creating interactive web applications.', 'Algorithms are self-contained sequences that carry out a variety of tasks.', 'Amp is a non-blocking concurrency library for PHP.', 'Android is an operating system built by Google designed for mobile devices.']


In [58]:
topic_url=[]

for url in topics_url:
    topic_url.append("https://github.com" + url['href'])
print(topic_url[:5])

['https://github.com/topics/3d', 'https://github.com/topics/ajax', 'https://github.com/topics/algorithm', 'https://github.com/topics/amphp', 'https://github.com/topics/android']


### Create a Dictionary for saving scraped data from Main Topics

In [63]:
topic_dict={'topic': topic_title, 
'topic_desc': topic_desc, 
'topic_url': topic_url}

topic_df= pd.DataFrame(topic_dict)

In [65]:
topic_df.head()

,topic,topic_desc,topic_url
0,3D,3D modeling is the process of virtually develo...,https://github.com/topics/3d
1,Ajax,Ajax is a technique for creating interactive w...,https://github.com/topics/ajax
2,Algorithm,Algorithms are self-contained sequences that c...,https://github.com/topics/algorithm
3,Amp,Amp is a non-blocking concurrency library for ...,https://github.com/topics/amphp
4,Android,Android is an operating system built by Google...,https://github.com/topics/android


In [67]:
# Write main_topic_list file in CSV
topic_df.to_csv('main_topic_list.csv', index=None)

### Now Scrap Username, Repository Name , Repository URL and Stars from each Topic Page

In [72]:
topic_page_url=topic_url[0]

In [73]:
topic_page_url

'https://github.com/topics/3d'

In [74]:
response= requests.get(topic_page_url)

In [79]:
response.status_code

200

In [78]:
soup= BeautifulSoup(response.text, 'html.parser')

In [84]:
user_name= soup.find_all('h3', {'class': 'f3 color-fg-muted text-normal lh-condensed'})

In [93]:
a_tags=user_name[0].find_all('a')

In [97]:
a_tags[0].text.strip()

'mrdoob'

In [100]:
a_tags[1].text.strip()

'three.js'

In [102]:
base_url="https://github.com"
repo_url= base_url + a_tags[1]['href']
print (repo_url)

https://github.com/mrdoob/three.js


In [111]:
stars=soup.find_all('span', {'class': 'Counter js-social-count'})

In [115]:
stars[0].text

'85.6k'

In [120]:
def parse_stars_count(stars_str):
    if (stars_str[-1] == 'k'):
        return int (float(k[:-1]) * 1000)
    return int(stars_str)

In [122]:
parse_stars_count(stars[0].text)


85600

### Encapsule all working for scraping Username, Repo Name , Repo URL in function

In [178]:
def get_topic_page (page_url):
    response= requests.get(page_url)
    if response.status_code != 200:
        raise Exception('Failed to load Page  '.format(page_url))
    
    soup= BeautifulSoup(response.text, 'html.parser')

    return soup

def get_repo_info(user_tag, stars_tag):
    a_tags= user_tag.find_all('a')
    user_name=a_tags[0].text.strip()
    repo_name=a_tags[1].text.strip()
    repo_url1=base_url + a_tags[1]['href']
    stars_tag= parse_stars_count(stars_tag.text)
    return  user_name, repo_name, repo_url1, stars_tag

def get_topic_repos (soup):
    user_name= soup.find_all('h3', {'class': 'f3 color-fg-muted text-normal lh-condensed'})
    stars=soup.find_all('span', {'class': 'Counter js-social-count'})
    
    topic_repo_dict={
    "user_name": [],
    "Repo_name": [],
    "Repo_URL": [],
    "Repo_stars": []
}

    for i in range (len(user_name)):
        repo_info= get_repo_info(user_name[i], stars[i])
        topic_repo_dict['user_name'].append(repo_info[0])
        topic_repo_dict['Repo_name'].append(repo_info[1])
        topic_repo_dict['Repo_URL'].append(repo_info[2])
        topic_repo_dict['Repo_stars'].append(repo_info[3])
        
    return pd.DataFrame (topic_repo_dict)

### Testing the functions

In [185]:
topic_url[6]

'https://github.com/topics/ansible'

In [186]:
get_topic_repos(get_topic_page(topic_url[6]))

,user_name,Repo_name,Repo_URL,Repo_stars
0,ansible,ansible,https://github.com/ansible/ansible,85600
1,bregman-arie,devops-exercises,https://github.com/bregman-arie/devops-exercises,85600
2,trailofbits,algo,https://github.com/trailofbits/algo,85600
3,StreisandEffect,streisand,https://github.com/StreisandEffect/streisand,85600
4,MichaelCade,90DaysOfDevOps,https://github.com/MichaelCade/90DaysOfDevOps,85600
5,kubernetes-sigs,kubespray,https://github.com/kubernetes-sigs/kubespray,85600
6,ansible,awx,https://github.com/ansible/awx,85600
7,easzlab,kubeasz,https://github.com/easzlab/kubeasz,85600
8,geerlingguy,ansible-for-devops,https://github.com/geerlingguy/ansible-for-devops,85600
9,khuedoan,homelab,https://github.com/khuedoan/homelab,85600


### Save all Topic Scalped Repository data in Data folder in CSV format

In [214]:

os.makedirs('data', exist_ok=True)
for i in range(len(topic_url)):
    fname= topic_title[i] + '.csv'
    if os.path.exists(fname):
        print("The file already Exist".format(fname))
    topics_repos=get_topic_repos(get_topic_page(topic_url[i]))
    topics_repos.to_csv('data/' + fname, index=None)

